## Lookuptable experiment

In [1]:
%load_ext autoreload
%autoreload 2
%env CUDA_VISIBLE_DEVICES=3
!echo $CUDA_VISIBLE_DEVICES

env: CUDA_VISIBLE_DEVICES=3
3


In [2]:
## discovery cluster setup
# CACHE_DIR = "/scratch/lucchetti.f/models/"
# os.environ['TRANSFORMERS_CACHE'] = CACHE_DIR

import os
import sys
sys.path.append('..')

import random
import gc
import torch
import pandas as pd
from model_utils import *
import tqdm as notebook_tqdm
from trace_model import ModelLoader
from pandas import DataFrame
torch.set_grad_enabled(False)
from tqdm import tqdm 
from model_utils import layername
import numpy as np
from extract_data import *
# import nltk
# nltk.download('words')
from random import sample
from patch import *

import re
check_devs()
# clear_devs()
torch.cuda.is_available()

MODEL_NAME = "bigcode/starcoder"

/home/franlucc/mechinterp/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


0 / 84978434048 used for device 0, reserved 0


In [3]:
lm = ModelLoader(MODEL_NAME)
check_devs()

Loading checkpoint shards: 100%|██████████| 7/7 [00:26<00:00,  3.81s/it]


62136934400 / 84978434048 used for device 0, reserved 62138613760


## synthetic look-up table

In [4]:

example_prompt = """def func(i : int) -> str:
    if i == 0:
        return "cat"
    elif i == 1:
        return "speed"
    elif i == 3:
        return "city"
    else:
        return "number"
"""

def make_assert(n):
    return f"assert func({n}) == \""

def layernum(lname):
    return int(lname.split(".")[-1])

### Step 1: prune transformer, get earliest layer that successfully completes task

problem: sampling is fast, greedy is slow

In [5]:
txt, ret_dict = lm.trace_generate(example_prompt + "\n" + make_assert(99),
                                  max_new_toks=1,
                                  request_logits=[layername(lm.model, l) for l in range(10,30)],
                                  do_greedy_decoding=True)
out = txt.replace(example_prompt + "\n" + make_assert(99), "")


In [6]:
print(out)
ret_dict["logits"]

number


{'transformer.h.10': [('1', 0.07470065),
  (' ', 0.030268962),
  ('0', 0.028924687),
  ('2', 0.022495646),
  ('x', 0.02022626),
  ('s', 0.019504828),
  ('<fim_suffix>', 0.011841485),
  ('a', 0.010499627),
  ('(', 0.009234566),
  ('\n', 0.008369252)],
 'transformer.h.11': [('1', 0.07264306),
  (' ', 0.040853888),
  ('0', 0.028550722),
  ('s', 0.023434542),
  ('x', 0.022879217),
  ('2', 0.020956304),
  ('a', 0.012987624),
  ('3', 0.01026766),
  ('<fim_suffix>', 0.009816614),
  ('(', 0.009485407)],
 'transformer.h.12': [('1', 0.04846015),
  ('x', 0.025483083),
  ('0', 0.021049295),
  (' ', 0.020173019),
  ('2', 0.015108691),
  ('s', 0.012573922),
  ('a', 0.010190361),
  ('3', 0.008590823),
  ('<fim_suffix>', 0.0072472095),
  ('T', 0.0071477443)],
 'transformer.h.13': [('1', 0.04847688),
  ('x', 0.030256456),
  (' ', 0.018648338),
  ('2', 0.016423674),
  ('0', 0.015762946),
  ('s', 0.013428384),
  ('a', 0.010456596),
  ('3', 0.0098661315),
  ('y', 0.008077291),
  ('T', 0.0065337443)],
 'tr

In [7]:
def trace_lookuptable(max_n, prompt, sample_from=None):
    res = []
    exec(prompt)
    for n in range(0, max_n):
        # for k in range(0, 20):
            txt, ret_dict = lm.trace_generate(prompt + "\n" + make_assert(n),
                                            max_new_toks=1,
                                            request_logits=[layername(lm.model, l) for l in range(10,30)],
                                            sample_from_topk_logits=sample_from)
            out = txt.replace(prompt + "\n" + make_assert(n), "")
            for lname, logits in ret_dict["logits"].items():
                if out in [str(x) for (x,_) in logits]:
                    gold = eval(f"func({n})")
                    res.append(((out == gold), out, gold, n, layernum(lname)))
                    break
    return res

## Try to break lookup table task

make harder by increasing lenght. At what point is transformer unsure what task this is?

In [8]:
def save_to_csv(res, name, header=['prompt_seed','is_correct','completion', 'gold', 'input n', 'layernum']):
    ds = DataFrame.from_dict(res)
    ds.to_csv(f"../results/{name}.csv", header=header)

In [9]:
def make_lookup_table(n, seed=0):
    assert n < 25, "n must be less than 25"
    random.seed(seed)
    nums = sample(list(range(n)), n)
    words = sample(list(range(n+1)), n+1)
    s = f"def func(i : int) -> str:\n\tif i == {nums[0]}:\n\t\treturn \"{chr(97+words[0])}\"\n"
    for i,k in enumerate(nums[1:]):
        s += f"\telif i == {k}:\n\t\treturn \"{chr(97+words[i+1])}\"\n"
    s += f"\telse:\n\t\treturn \"{chr(97+words[-1])}\""
    return s.replace("\t","    ")

In [10]:
def trace_false_task(max_n, prompt, seed, len_table=12):
    res = []
    exec(prompt)
    
    for n in range(0, max_n):
        # for k in range(0, 20):
            txt, ret_dict = lm.trace_generate(prompt + "\n" + make_assert(n),
                                            max_new_toks=1,
                                            request_logits=[layername(lm.model, l) for l in range(10,20)],
                                            do_greedy_decoding=True)
            out = txt.replace(prompt + "\n" + make_assert(n), "")
            for lname, logits in ret_dict["logits"].items():
                if out in [str(x) for (x,_) in logits]:
                    gold = eval(f"func({n})")
                    def first_or_last_match(out, i):
                        exec(prompt)
                        first_digit = eval(f"func({str(i)[0]})")
                        last_digit = eval(f"func({str(i)[-1]})")
                        if first_digit == out:
                            return "Match_First_Digit_False_Task"
                        elif last_digit == out:
                            return "Match_Last_Digit_False_Task"
                        else:
                            return "Neither_False_Task"
                    is_correct = (out == gold)
                    first_or_last_match_str = first_or_last_match(out, n) if not is_correct else "NA"
                    res.append((len_table, seed, is_correct, first_or_last_match_str, out, gold, n, layernum(lname)))
                    break
    return res

In [11]:
import json
def exp_false_task(seed_range, lengths_table=[3,6,9,12], seed_start=0):
    fp = open(f"../results/log.json", "w")
    tot_res = []
    for len_table in tqdm(lengths_table):
        for seed in range(seed_start, seed_range):
            prompt = make_lookup_table(len_table, seed=seed)
            res = trace_false_task(50, prompt, seed, len_table=len_table)
            json.dump(res, fp)
            tot_res += res
    fp.close()
    return tot_res


## Run exp

In [10]:
tot_res = exp_false_task(2)
save_to_csv(tot_res, "full_lookuptable_task0", header =["table_len","seed","is_correct","first_or_last_match","out","gold","func(n)","earliest_layer"])

100%|██████████| 4/4 [16:27<00:00, 246.77s/it]


In [11]:
tot_res = exp_false_task(20, seed_start=2)
save_to_csv(tot_res, "full_lookuptable_task1", header =["table_len","seed","is_correct","first_or_last_match","out","gold","func(n)","earliest_layer"])

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/4 [20:00<?, ?it/s]


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 tot_res = exp_false_task(20, seed_start=2)                                                   │
│   2 save_to_csv(tot_res, "1b_full_lookuptable_task1", header =["table_len","seed","is_correc     │
│   3                                                                                              │
│                                                                                                  │
│ in exp_false_task:8                                                                              │
│                                                                                                  │
│    5 │   for len_table in tqdm(lengths_table):                                                   │
│    6 │   │   for seed in range(seed_start, seed_range):                                          │
│    7 │   │   │   prompt = make_lookup_table(len_table, seed=seed)                                │
│ ❱  8 │   │   │   res = trace_false_task(50, prompt, seed, len_table=len_table)                   │
│    9 │   │   │   json.dump(res, fp)                                                              │
│   10 │   │   │   tot_res += res                                                                  │
│   11 │   fp.close()                                                                              │
│                                                                                                  │
│ in trace_false_task:7                                                                            │
│                                                                                                  │
│    4 │                                                                                           │
│    5 │   for n in range(0, max_n):                                                               │
│    6 │   │   # for k in range(0, 20):                                                            │
│ ❱  7 │   │   │   txt, ret_dict = lm.trace_generate(prompt + "\n" + make_assert(n),               │
│    8 │   │   │   │   │   │   │   │   │   │   │   max_new_toks=1,                                 │
│    9 │   │   │   │   │   │   │   │   │   │   │   request_logits=[layername(lm.model, l) for l    │
│   10 │   │   │   │   │   │   │   │   │   │   │   do_greedy_decoding=True)                        │
│                                                                                                  │
│ /home/franlucc/codetrace/notebooks/../trace_model.py:215 in trace_generate                       │
│                                                                                                  │
│   212 │   │   │   │                                                                              │
│   213 │   │   │   │   # collect tokens generated from final layer                                │
│   214 │   │   │   │   for i in range(input_ids.size(0)):                                         │
│ ❱ 215 │   │   │   │   │   generated_tokens[i].append(                                            │
│   216 │   │   │   │   │   │   [                                                                  │
│   217 │   │   │   │   │   │   │   {"token": self.tokenizer.decode(t), "id": t.item(), "p": sof   │
│   218 │   │   │   │   │   │   │   for t in topk_logits[i]                                        │
│                                                                                                  │
│ /home/franlucc/codetrace/notebooks/../trace_model.py:216 in <listcomp>                           │
│                                                                                                  │
│   213 │   │   │   │   # collect tokens generated from final

In [ ]:
tot_res = exp_false_task(20)
save_to_csv(tot_res, "1b_lookup_table_false_task_small0")

## Patch the hyp layer that leads to false task

layer 10

## Noising:

In [12]:
heads_to_noise = [(h, 18) for h in range(48)]
## noise create
uniform_noise = False
noise = 0.1
replace=True
rs = numpy.random.RandomState(1)  # For reproducibility, use pseudorandom noise
if uniform_noise:
    prng = lambda *shape: rs.uniform(-1, 1, shape)
else:
    prng = lambda *shape: rs.randn(*shape)
if isinstance(noise, float):
    noise_fn = lambda x: noise * x
else:
    noise_fn = noise
    
heads_to_noise = [(head, layername(lm.model, layer)) for (head, layer) in heads_to_noise]
head_dim = lm.model.config.n_embd // lm.model.config.n_head
def patch_rep(x, layer): # x is the output of the attn forward method (layer), layer is layer num
    ## x is tuple (attn_weights, present[key value tracker for MQA], opt(attn_weights))
    if layer in [layer for (_, layer) in heads_to_noise]:
        for head in [head for (head, layer_) in heads_to_noise if layer_ == layer]:
            print("patching (layer, head): ", layer, head)
            ## noise at the corresponding heads
            start = head_dim * head
            end = start + head_dim
            noise_data = noise_fn(
                    torch.from_numpy(prng(x[0].shape[0], x[0].shape[1], head_dim))
                ).to(x[0].device)
            if replace:
                x[0][:,:,start:end] = noise_data
            else:
                x[0][:,:,start:end] += noise_data
    return x


## Masking:

try to fix example from `full_lookuptable_task1.csv`

`idx,table_len,seed,is_correct,first_or_last_match,out,gold,func(n),earliest_layer`
`473,3,11,False,Match_First_Digit_False_Task,a,c,23,10`

In [12]:
txt, ret_dict = lm.trace_generate(make_lookup_table(3, 11) + "\n" + make_assert(23), max_new_toks=1)

In [13]:
txt

'def func(i : int) -> str:\n    if i == 1:\n        return "b"\n    elif i == 2:\n        return "a"\n    elif i == 0:\n        return "d"\n    else:\n        return "c"\nassert func(23) == "a'

In [24]:
print(make_lookup_table(3, 11))

def func(i : int) -> str:
    if i == 1:
        return "b"
    elif i == 2:
        return "a"
    elif i == 0:
        return "d"
    else:
        return "c"


In [50]:
df = pd.read_csv("../results/full_lookuptable_task1.csv")
df = df.loc[df["is_correct"] == False]
df = df.loc[df["table_len"] <= 9]
df["patch_18_res"] = ""
df


,Unnamed: 0,table_len,seed,is_correct,first_or_last_match,out,gold,func(n),earliest_layer,patch_18_res
21,21,3,2,False,Match_First_Digit_False_Task,a,d,21,10,
22,22,3,2,False,Match_First_Digit_False_Task,a,d,22,10,
24,24,3,2,False,Match_First_Digit_False_Task,a,d,24,10,
25,25,3,2,False,Match_First_Digit_False_Task,a,d,25,10,
26,26,3,2,False,Match_First_Digit_False_Task,a,d,26,10,
...,...,...,...,...,...,...,...,...,...,...
2695,2695,9,19,False,Match_First_Digit_False_Task,b,f,45,18,
2696,2696,9,19,False,Match_First_Digit_False_Task,b,f,46,18,
2697,2697,9,19,False,Match_First_Digit_False_Task,b,f,47,18,
2698,2698,9,19,False,Match_First_Digit_False_Task,b,f,48,18,


In [60]:
from patch import move_hidden_states

for row in df.iterrows():
    table_len = row[1]["table_len"]
    seed = row[1]["seed"]
    n = row[1]["func(n)"]
    prompt = make_lookup_table(table_len, seed=seed) + "\n" + make_assert(n)
    out, probs = move_hidden_states(lm,
                                [prompt],
                               [(6,10)],
                                pick_greedily=True,)
    out = lm.tokenizer.decode(out[0])
    if out == row[1]["gold"]:
        df.loc[row[0], "patch_18_res"] = "Success"
    elif out == row[1]["out"]:
        df.loc[row[0], "patch_18_res"] = "Failure"
    else:
        df.loc[row[0], "patch_18_res"] = out

In [61]:
len(df.loc[df["patch_18_res"] == "Success"]), len(df.loc[df["patch_18_res"] == "Failure"]), len(df)

(424, 135, 808)

In [62]:
df3 = df.loc[df["table_len"] == 3]
print(len(df3.loc[df3["patch_18_res"] == "Success"])/len(df3))
df6 = df.loc[df["table_len"] == 6]
print(len(df6.loc[df6["patch_18_res"] == "Success"])/len(df6))
df9 = df.loc[df["table_len"] == 9]
print(len(df9.loc[df9["patch_18_res"] == "Success"])/len(df9))

0.8768115942028986
0.27680798004987534
0.7137546468401487


In [22]:
lm.tokenizer.decode(out[0])

'c'

In [16]:
layer_src = layername(lm.model, 8)
layer_dst = layername(lm.model, 10)

saved_layer_output = []
def patch_rep(x, layer):
    """
    x is the output of the forward method (either attn, mlp, or block)
    """
    print(layer, layer_src, layer_dst)
    if layer == layer_src:
        saved_layer_output = x
        assert all(torch.eq(saved_layer_output[0], x[0]).flatten().tolist())
        print(f"saved layer output {layer_src}")
        return x
    elif layer == layer_dst:
        return saved_layer_output
    else:
        return x

In [17]:
tokenized = lm.tokenizer(make_lookup_table(3, 11) + "\n" + make_assert(23), 
                         padding=True, 
                         return_tensors="pt").to(lm.model.device)
input_ids, attention_mask = tokenized["input_ids"], tokenized["attention_mask"]

In [18]:
with TraceDict(
    lm.model, 
    layers = [layername(lm.model, i) for i in range(1, 40)], 
    retain_input=True,
    retain_output=True,
    edit_output_attn=patch_rep,
) as traces:
    model_out = lm.model(
        input_ids=input_ids,
        attention_mask=attention_mask,
    )

transformer.h.1 transformer.h.8 transformer.h.10
transformer.h.2 transformer.h.8 transformer.h.10
transformer.h.3 transformer.h.8 transformer.h.10
transformer.h.4 transformer.h.8 transformer.h.10
transformer.h.5 transformer.h.8 transformer.h.10
transformer.h.6 transformer.h.8 transformer.h.10
transformer.h.7 transformer.h.8 transformer.h.10
transformer.h.8 transformer.h.8 transformer.h.10
saved layer output transformer.h.8
transformer.h.9 transformer.h.8 transformer.h.10
transformer.h.10 transformer.h.8 transformer.h.10


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:8                                                                                    │
│                                                                                                  │
│    5 │   retain_output=True,                                                                     │
│    6 │   edit_output_attn=patch_rep,                                                             │
│    7 ) as traces:                                                                                │
│ ❱  8 │   model_out = lm.model(                                                                   │
│    9 │   │   input_ids=input_ids,                                                                │
│   10 │   │   attention_mask=attention_mask,                                                      │
│   11 │   )                                                                                       │
│                                                                                                  │
│ /home/franlucc/mechinterp/lib/python3.10/site-packages/torch/nn/modules/module.py:1501 in        │
│ _call_impl                                                                                       │
│                                                                                                  │
│   1498 │   │   if not (self._backward_hooks or self._backward_pre_hooks or self._forward_hooks   │
│   1499 │   │   │   │   or _global_backward_pre_hooks or _global_backward_hooks                   │
│   1500 │   │   │   │   or _global_forward_hooks or _global_forward_pre_hooks):                   │
│ ❱ 1501 │   │   │   return forward_call(*args, **kwargs)                                          │
│   1502 │   │   # Do not call functions when jit is used                                          │
│   1503 │   │   full_backward_hooks, non_full_backward_hooks = [], []                             │
│   1504 │   │   backward_pre_hooks = []                                                           │
│                                                                                                  │
│ /home/franlucc/mechinterp/lib/python3.10/site-packages/transformers/models/gpt_bigcode/modeling_ │
│ gpt_bigcode.py:808 in forward                                                                    │
│                                                                                                  │
│    805 │   │   """                                                                               │
│    806 │   │   return_dict = return_dict if return_dict is not None else self.config.use_return  │
│    807 │   │                                                                                     │
│ ❱  808 │   │   transformer_outputs = self.transformer(                                           │
│    809 │   │   │   input_ids,                                                                    │
│    810 │   │   │   past_key_values=past_key_values,                                              │
│    811 │   │   │   attention_mask=attention_mask,                                                │
│                                                                                                  │
│ /home/franlucc/mechinterp/lib/python3.10/site-packages/torch/nn/modules/module.py:1501 in        │
│ _call_impl                                                                                       │
│                                                                                                  │
│   1498 │   │   if not (self._backward_hooks or self._backward_pre_hooks or self._forward_hooks   │
│   1499 │   │   │   │   or _global_backward_pre_hooks or _global_backward_hooks                   │
│   1500 │   │   │   │   or _global_forward_hooks or _global_forward_pre_hooks):                   │
│ ❱ 1501 │   │   │   return forward_call(*args, **kwargs)    

## Grab from other run

testing layer x from promtp a -> layer y from prompt b

In [104]:
prompt_a = make_lookup_table(3, 5) + "\n\n" + make_assert(2)
print(prompt_a)

def func(i : int) -> str:
    if i == 0:
        return "d"
    elif i == 1:
        return "b"
    elif i == 2:
        return "a"
    else:
        return "c"

assert func(2) == "


In [105]:
prompt_b = make_lookup_table(3, 4) + "\n\n" + make_assert(1)
print(prompt_b)

def func(i : int) -> str:
    if i == 0:
        return "d"
    elif i == 1:
        return "b"
    elif i == 2:
        return "a"
    else:
        return "c"

assert func(1) == "


In [106]:
from patch import patch_a_b_layers

out, probs = patch_a_b_layers(lm,
                            prompt_a,
                            prompt_b,
                            [(10,10)],
                            pick_greedily=True,)

lm.tokenizer.decode(out[0])

'a'